# Поймай меня, если сможешь

https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2

Будем решать задачу идентификации взломщика по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в статье на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

В этом соревновании будем решать похожую задачу: алгоритм будет анализировать последовательность из нескольких веб-сайтов, посещенных подряд одним и тем же человеком, и определять, Элис это или взломщик (кто-то другой).

Данные собраны с прокси-серверов Университета Блеза Паскаля. "A Tool for Classification of Sequential Data", авторы Giacomo Kahn, Yannick Loiseau и Olivier Raynaud.

In [1]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

import seaborn as sns
from matplotlib import pyplot as plt
import os
os.chdir("/Users/iakubovskii/Machine_Learning/RANEPA/Fintech_2020/Машинное обучение/Данные/Alice/")
import warnings
warnings.filterwarnings("ignore")

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv("train_sessions.csv.zip", index_col="session_id")
test_df = pd.read_csv("test_sessions.csv.zip", index_col="session_id")

# приведем колонки time1, ..., time10 к временному формату
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by="time1")

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:

- site1 – индекс первого посещенного сайта в сессии
- time1 – время посещения первого сайта в сессии
- ...
- site10 – индекс 10-го посещенного сайта в сессии
- time10 – время посещения 10-го сайта в сессии
- target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей

Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [4]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# загрузим словарик сайтов
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [7]:
# наша целевая переменная
y_train = train_df["target"]

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [12]:
print(f"Сессии Алисы из тренировочной выборки : {(y_train == 1).sum()}", 
      f"Сессии не-Алисы из тренировочной выборки : {(y_train == 0).sum()}")

Сессии Алисы из тренировочной выборки : 2297 Сессии не-Алисы из тренировочной выборки : 251264


Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея: у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.

Подготовим данные, из всей таблицы выберем только признаки site1, site2, ... , site10. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [13]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается примерно вот так:

In [18]:
from scipy.sparse import csr_matrix

def create_csr(full_sites):
    # последовательность с индексами
    sites_flatten = full_sites.values.flatten()

    # искомая матрица
    full_sites_sparse = csr_matrix(
        (
            [1] * sites_flatten.shape[0],
            sites_flatten,
            range(0, sites_flatten.shape[0] + 10, 10),
        )
    )[:, 1:]
    return(full_sites_sparse)
full_sites_sparse = create_csr(full_sites)

**Преимущества формата CSR**
  - эффективные арифметические операции CSR + CSR, CSR * CSR и т.д.
  - эффективная нарезка строк
  - быстрые матрично-векторные произведения

**Недостатки формата CSR**
  - медленные операции нарезки столбцов (рассмотрим CSC)
  - изменение структуры разреженности требует больших затрат (рассмотрите LIL или DOK)

In [19]:
full_sites_sparse

<336358x48371 sparse matrix of type '<class 'numpy.int64'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [29]:
sites_flatten = full_sites.values.flatten()
print(np.sum(sites_flatten == 0)) 
print(np.sum(sites_flatten != 0)) # 3195430 stored elements

168150
3195430


In [123]:
full_sites.values.max() # 336358x48371

48371


Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

## Первая модель

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией логистической регрессии из пакета sklearn с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation).

**Напишем простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучим наш первый классификатор**

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MaxAbsScaler
min_max_sc = MaxAbsScaler()

def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=17):
    """
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=ratio,
                                                        random_state=seed)
    X_train_sc = min_max_sc.fit_transform(X_train)
    X_test_sc = min_max_sc.transform(X_test)
    lr = LogisticRegression(C=C)
    lr.fit(X_train_sc, y_train)
    y_predict = lr.predict(X_test_sc)
    return roc_auc_score(y_test, y_predict)

In [34]:
roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f"ROC AUC на отложенной выборке = {roc_auc}")

ROC AUC на отложенной выборке = 0.5117612904188056


Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке необходимо обучить модель заново уже на всей обучающей выборке (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

## Улучшаем модель

Создадим такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный линейный тренд за весь период предоставленных данных.

In [36]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
full_sites['session_date'] = full_df['time1'].dt.year.apply(str) +  \
                          full_df['time1'].dt.month.apply(str).apply(lambda x: "0" + x if len(x)==1 else x)
full_sites['session_date'] = full_sites['session_date'].apply(np.int32)

from scipy.sparse import hstack
full_sites_sparse = hstack([full_sites_sparse, 
                            ohe.fit_transform(full_sites[['session_date']])])
full_sites_sparse = csr_matrix(full_sites_sparse)

In [37]:
full_sites_sparse

<336358x48395 sparse matrix of type '<class 'numpy.float64'>'
	with 2203256 stored elements in Compressed Sparse Row format>

Добавим новый признак и снова посчитаем ROC AUC на отложенной выборке.

In [38]:
roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f"ROC AUC на отложенной выборке = {roc_auc}")

ROC AUC на отложенной выборке = 0.5220645785844671


In [168]:
full_sites_sparse.shape

(336358, 48394)

Добавим два новых признака: *start_hour* и *morning*.

Признак *start_hour* – это час в который началась сессия (от 0 до 23), а бинарный признак *morning* равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если start_hour равен 11 или меньше).

Посчитйте ROC AUC на отложенной выборке для выборки с:

- сайтами, start_month и start_hour

- сайтами, start_month и morning

- сайтами, start_month, start_hour и morning

In [44]:
full_sites['start_hour'] = full_df['time1'].dt.hour
full_sites['start_month'] = full_df['time1'].dt.month
full_sites['mornning'] = full_sites['start_hour'].apply(lambda x: 1 if x <=11 else 0) 

from scipy.sparse import hstack
full_sites_sparse = hstack([full_sites_sparse, 
                            ohe.fit_transform(full_sites[['start_hour']]),
                            ohe.fit_transform(full_sites[['start_month']]),
                           ohe.fit_transform(full_sites[['mornning']])])
full_sites_sparse = csr_matrix(full_sites_sparse)

In [50]:
roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f"ROC AUC на отложенной выборке = {roc_auc}")

ROC AUC на отложенной выборке = 0.5741407113711351


## Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейзлайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитаем качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию C=1.

Теперь погнали искать оптимальный гиперпараметр

In [61]:
C = np.linspace(0.01, 10, 10)
results_C = dict(zip(C, list(map(lambda x: get_auc_lr_valid(full_sites_sparse[:idx_split], y_train, C=x),
                            C
                           )
                            )
                    )
                )
# roc_auc = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
# print(f"ROC AUC на отложенной выборке = {roc_auc}")

In [62]:
results_C

{1.0: 0.5741407113711351,
 2.0: 0.6116436416824489,
 3.0: 0.6331662141983105,
 4.0: 0.6458700296362873,
 5.0: 0.6560959420342706,
 6.0: 0.6641447993655248,
 7.0: 0.6677886996302755,
 8.0: 0.6726715514150227,
 9.0: 0.6746008161777524,
 10.0: 0.6785146141086962}

## 1.4 Использование LASSO-регрессии для выкидывания ненужных признаков

## 1.5 Для того, чтобы повысить точность, делайте, что хотите 

https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/code